In [ ]:
# import matplotlib.pylab as plt
%pylab inline

In [ ]:
from pandas.tools.plotting import scatter_matrix

In [ ]:
import datetime
import glob
import re
import time

In [ ]:
import pandas as pd

In [ ]:
# Number of lines containing ALL the jobs, 6898016
# Number of lines after filtering INVALID jobs, 2591267
perc_valid = 2591267./6898016.
print perc_valid
print "User's have to launch a job 3 times in average"

In [ ]:
# Almost 40% of the jobs this year have been invalid

In [ ]:
myfields = [u'JobID', u'JobIDRaw', u'AllocCPUS', u'AveCPU', u'AveDiskRead', u'AveDiskWrite',
                   u'AvePages', u'AveRSS', u'Comment', u'CPUTime', u'CPUTimeRAW',
                   u'DerivedExitCode', u'Elapsed', u'Eligible', u'End', u'ExitCode',
                   u'MaxDiskRead', u'MaxDiskReadNode', u'MaxDiskReadTask', u'MaxDiskWrite',
                   u'MaxDiskWriteNode', u'MaxDiskWriteTask', u'MaxPages', u'MaxPagesNode',
                   u'MaxPagesTask', u'MaxRSS', u'MaxRSSNode', u'MaxRSSTask', u'MaxVMSize',
                   u'MaxVMSizeNode', u'MaxVMSizeTask', u'MinCPU', u'MinCPUNode', u'MinCPUTask',
                   u'NCPUS', u'NNodes', u'NTasks', u'Partition', u'ReqCPUS', u'ReqMem',
                   u'Start', u'State', u'Submit', u'SystemCPU', u'Timelimit', u'TotalCPU',
                   u'User', u'UserCPU']

In [ ]:
#myfields

In [ ]:
data = pd.read_csv('./2015-2016_valid.csv', delimiter = '|', usecols=myfields, low_memory=False)
data = data[~data.User.isnull()]
original_data = data
data = data[(data.Partition == 'tgac-short') | (data.Partition == 'tgac-medium') | (data.Partition == 'tgac-long')]

In [ ]:
# print data.columns
# print data.shape

In [ ]:
data.AllocCPUS.unique()

In [ ]:
data = original_data[(original_data.Partition == 'tgac-short') | (original_data.Partition == 'tgac-medium') | (original_data.Partition == 'tgac-long')]

In [ ]:
#data = read_data

In [ ]:
# data.ReqMem.unique()

In [ ]:
original_data = original_data[original_data.Partition == 'tgac-seed']

In [ ]:
original_data.User.unique()

In [ ]:
monthly_mean = data[['User']].groupby([data.Submit.map(lambda x: datetime.datetime.strptime("%s-%s" %(x.year, x.month), "%Y-%m"))]).agg({'User': pd.Series.nunique})

In [ ]:
monthly_mean.columns

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(monthly_mean)
plt.ylabel("# Distinct Users")
plt.xticks(rotation=70)
plt.show()

In [ ]:
def parse_date(duration):
    sline = duration.split("-")
    if len(sline)>1:
        hh = sline[1].split(":")
        hours = int(sline[0])*24 + int(hh[0]) + (float(hh[1])/60) 
    elif len(sline) == 1:
        hh = sline[0].split(":")
        if len(hh) == 1:
            return 0
        else:
            hours = int(hh[0]) + (float(hh[1])/60) 
    else:
        hours = 0
    return hours

def parse_memory(x):
    cpus = x['NCPUS']
  
    memlist = re.split("([M,c,G,T,n])", x['ReqMem'])
    mem = memlist[0]
    units = memlist[1]
    if 'c' in units:
        tot_mem = cpus*float(mem)
    else:
        tot_mem = float(mem)
    
    if 'G' in x['ReqMem']:
        tot_mem *= 1
    elif 'T' in x['ReqMem']:
        tot_mem *= 1000
    else:
        tot_mem *= 0.001
    return tot_mem

def parse_date_end(s):
    if s == "Unknown":
        date = np.nan
    else:
        date = datetime.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return date

def parse_maxRss(x):
    if x != np.nan and type(x) == str:
        mem = float(x.strip('K'))*0.000001
    else:
        mem = 0
    return mem

In [ ]:
data['tiempo'] = data['Elapsed'].apply(parse_date)
data['RQTMem'] = data.apply(parse_memory, axis=1)
data['Start'] = data.Start.apply(parse_date_end)
data['Submit'] = data.Submit.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))
data['End'] = data.End.apply(parse_date_end)
data['MaxRSS'] = data.MaxRSS.apply(parse_maxRss)
data['AllocCPUS'] = data.AllocCPUS.apply(pd.to_numeric)

In [ ]:
data['Comment'] = 1
data['RExitCode'] = data.ExitCode.apply(lambda x: int(x.split(':')[0]) if x.split(':')[0] > 127 else int(x.split(':')[0])-128)

In [ ]:
udata = data[data.Start.notnull() & data.End.notnull()]

In [ ]:
print len(data)
print data[(data.RQTMem<256)].RQTMem.count()

In [ ]:
series = data[data.RQTMem<256].RQTMem
fig, ax = plt.subplots()
series.hist(ax=ax, bins=100)
ax.set_yscale('log')

In [ ]:
series = data[data.RQTMem<126].RQTMem
plt.figure(figsize=(10,10))
series.hist(log=True, bins=25)
plt.xticks(np.arange(min(series), max(series)+1, 8.0));
plt.ylabel("Log(Frequency)")
plt.xlabel("Requested RAM")
plt.xlim([0,130]);

In [ ]:
series = data[(data.RQTMem>128) & (data.RQTMem<500)].RQTMem
plt.figure(figsize=(10,10))
series.hist(log=True, bins=25)
#plt.xticks(np.arange(min(series), max(series)+1, 8.0));
plt.ylabel("Log(Frequency)")
plt.xlabel("Requested RAM > 128")

In [ ]:
## filter user
# udata = data[data.User=='yanesl']
#udata = data

# get monthly data
monthly_mean = udata[['RQTMem', 'MaxRSS']].groupby([udata.Submit.map(lambda x: datetime.datetime.strptime("%s-%s" %(x.year, x.month), "%Y-%m"))]).agg([np.mean, np.sum])

In [ ]:
monthly_mean.RQTMem

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(monthly_mean.RQTMem['sum']/1344000. * 100, label="RAM Requested")
plt.axhline(y=100, color="red", label="RAM Available")
plt.ylabel("Requested/Available ratio in %")
plt.xticks(rotation=70)
plt.show()


In [ ]:
print "Monthly maximum RSS"
print monthly_mean.MaxRSS['sum']
print "\n\nMonthly RSS/available rate"
print monthly_mean.MaxRSS['sum']/1344000.

In [ ]:
#data[data.Start>datetime.datetime(2016,8,1)]["MaxRSS"].unique()

In [ ]:
# print dialy_mean.columns
fig = plt.figure(figsize=(10,10))
#plt.plot(monthly_mean.RQTMem['sum'].index, monthly_mean.MaxRSS['sum'].values, label="RSS")
plt.plot(monthly_mean.RQTMem['sum'].index, monthly_mean.RQTMem['sum'].values, label="Requested")
plt.axhline(y=1344000, color="red", label="Max Avail")
plt.xlabel("Month")
plt.ylabel("Log(Max RAM in GB)")
plt.legend(loc=0)
# ax.set_xscale("log")
plt.yscale("log")

In [ ]:
# There seems to be a 5 orders of magnitude difference between
# Requested and MaxRSS (problem with cgroups memory accounting?)
# If so, the magnitude of the problem is measured here.
# Also causing a 30% usage of the CPUs on the cluster...

# This is serious and there seems to be a patch on an available version of SLURM which diminishes this problem.

In [ ]:
# Please read:
# https://groups.google.com/forum/#!topic/slurm-devel/EElKUE0z4c0
# http://permalink.gmane.org/gmane.comp.distributed.slurm.devel/6921

# The patches NEED to be applied URGENTLY!

In [ ]:
data.AllocCPUS.unique()

In [ ]:
data[['AllocCPUS']] = data[['AllocCPUS']].apply(pd.to_numeric)

In [ ]:
data.AllocCPUS.unique()

In [ ]:
plt.figure(figsize=(10,10))
data[(data['tiempo']>.1) & (data['tiempo']<100)].tiempo.hist(bins=120)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
data[(data['tiempo']>2) & (data['tiempo']<100)].tiempo.hist(bins=120)
plt.show()

In [ ]:
#fdata = data[(data['tiempo']>0.) & (data['tiempo']<2.) & (data['AllocCPUS']>0) & (data['AllocCPUS']<32)]
fdata = data[(data['tiempo'] > 0.1) & (data['tiempo'] < 1000) & (data['AllocCPUS']< 128)]
print len(fdata)

In [ ]:
colormap = 'gnuplot'
plt.figure(figsize=(15, 5))
plt.subplot(1,2,1)
fdata = data[(data['tiempo'] > 0.1) & (data['tiempo'] < 100.) & (data['AllocCPUS'] < 32)]
plt.hexbin(fdata.tiempo.values, fdata.AllocCPUS.values, bins='log', gridsize=20, cmap=colormap)
plt.xlabel("Time (h)")
plt.ylabel("Cores (#)")
cb = plt.colorbar()
cb.set_label('log10(N)')
plt.subplot(1,2,2)
fdata = data[(data['tiempo'] > 2) & (data['tiempo'] < 100) & (data['AllocCPUS'] < 32)]
plt.hexbin(fdata.tiempo.values, fdata.AllocCPUS.values, bins='log', gridsize=20, cmap=colormap)
plt.xlabel("Time (h)")
plt.ylabel("Cores (#)")
cb = plt.colorbar()
cb.set_label('log10(N)')
plt.show()

In [ ]:
data.columns

In [ ]:
(data[data['RQTMem']<=256].RQTMem/1000).hist(bins=20)

In [ ]:
data.groupby(["User"]).sum().columns

In [ ]:
partitions = ['tgac-long', 'tgac-medium', 'tgac-short']
fig = plt.figure(figsize=(10,30))
cont = 1
for dp in partitions:
    df = data[(data.Partition == dp)]
    dd = df.groupby('User').mean()
    ax = plt.subplot(3, 1, cont)
    ax.set_title(dp)
    ax.plot(dd.AllocCPUS.values, dd.tiempo.values, 'o')
    for p in dd.itertuples():
        ax.annotate(p[0], xy=(p[1], p[8])) 
    for x in [1, 8, 16, 32, 64]:
        plt.axvline(x)
#     ax.set_xscale("log", nonposx='clip')    
#     ax.set_yscale("log", nonposx='clip')    
    ax.set_xlabel('Mean AllocCPUs')
    ax.set_ylabel('Mean time')
    cont += 1

In [ ]:
partitions = ['tgac-long', 'tgac-medium', 'tgac-short']
fig = plt.figure(figsize=(10,30))
cont = 1
for dp in partitions:
    df = data[(data.Partition == dp)]
    dd = df.groupby('User').sum()
    ax = plt.subplot(3, 1, cont)
    ax.set_title(dp)
    ax.plot(dd.AllocCPUS.values, dd.tiempo.values, 'o')
    for p in dd.itertuples():
        ax.annotate(p[0], xy=(p[1], p[8])) 
    ax.axhline(0.75)
    ax.axhline(6)
    ax.set_xscale("log", nonposx='clip')
    ax.set_yscale("log", nonposx='clip')    
    ax.set_xlabel('Total AllocCPUs')
    ax.set_ylabel('Total time')
    cont += 1

In [ ]:
partitions = ['tgac-long', 'tgac-medium', 'tgac-short']
fig = plt.figure(figsize=(10,30))
cont = 1
for dp in partitions:
    df = data[(data.Partition == dp)]
    dd = df.groupby('User').sum()
    dd['Ratio'] = dd['tiempo'] / dd['AllocCPUS']
    ax = plt.subplot(3, 1, cont)
    ax.set_title(dp)
    ax.plot(dd.AllocCPUS.values, dd.Ratio.values, 'o')
    for p in dd.itertuples():
        ax.annotate(p[0], xy=(p[1], p[8])) 
    ax.set_xscale("log", nonposx='clip')    
    ax.set_yscale("log", nonposx='clip')    
    ax.set_xlabel('Total AllocCPUs')
    ax.set_ylabel('Total Total time / Alloc cpus')
    cont += 1

In [ ]:
print (dd["tiempo"] / dd['AllocCPUS']).hist()

In [ ]:
df = data[(data.Partition == 'tgac-long') | (data.Partition == 'tgac-medium') | (data.Partition == 'tgac-short')]
# df = data[(data.Partition == 'tgac-long')]
dd = df.groupby('User').std()
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)
ax.plot(dd.AllocCPUS.values, dd.tiempo.values, 'o')
for p in dd.itertuples():
    ax.annotate(p[0], xy=(p[1], p[2])) 
# ax.set_xscale("log", nonposx='clip')    
# ax.set_yscale("log", nonposx='clip')    
ax.set_xlabel('Total AllocCPUs')
ax.set_ylabel('Total time')

In [ ]:
data['eff'] = data['AllocCPUS']/data['AllocCPUS']
adt = data.groupby('User').mean().sort('eff', ascending=False)
adt.sort()
print adt

In [ ]:
leyenda= []
for u in data.User.unique()[:10]:
    duag = data[data.User == u].groupby('month').sum()
    plt.plot(duag.index, duag.AllocCPUS.values, 'o-')
    leyenda.append(u)
    plt.xlabel('Mean AllocCPUs')
    plt.ylabel('Mean time')
plt.legend(leyenda, loc=0)

In [ ]:
data.columns

In [ ]:
#data.Submit.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))
data['year'], data['month'] = data['Start'].dt.year, data['Start'].dt.month

In [ ]:
data.columns

In [ ]:
for u in data.User.unique()[:10]:
    duag = data[data.User == u].groupby('month').sum()
    plt.figure()
    plt.title(u)
    plt.plot(duag.tiempo.values, duag.AllocCPUS.values, 'o-')
    for x, m in duag.iterrows():
        plt.annotate(x, xy=(m.tiempo, m.AllocCPUS))
    plt.xlabel('Mean AllocCPUs')
    plt.ylabel('Mean time')

In [ ]:
dd

In [ ]:

partitions = ['tgac-long', 'tgac-medium', 'tgac-short']
fig = plt.figure(figsize=(10,30))
cont = 1
for dp in partitions:
    df = data[(data.Partition == dp)]
    dd = df.groupby('month').mean()
    dd.columns
    ax = plt.subplot(3, 1, cont)
    ax.set_title(dp)
    ax.plot(dd.index, dd.tiempo)
    ax.plot(dd.index, dd.AllocCPUS)
    ax.set_xlabel('Month')
    ax.set_ylabel('CPUs')
    cont += 1
    ax.legend(['time', 'cpus'])

In [ ]:
cpu_usage = data.groupby('month').sum().AllocCPUS/24/28
print cpu_usage

In [ ]:
available_cpus_day = 93*32
print available_cpus_day

In [ ]:
alloc_cpus_day = 962.
float(alloc_cpus_day/available_cpus_day)

In [ ]:
available_mem_day = 13.44*1000
print available_mem_day, "TB"

In [ ]:
#df['ReqMem', 'AvailMem', 'ratio'] = data.groupby('month').agg(['sum']).RQTMem/4/7

In [ ]:
with pd.option_context('display.max_rows', 999, 'display.max_columns', 3):
    print data.groupby(['RExitCode', 'User']).agg(['count']).Comment

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(10,10))
ax = data.groupby(['RExitCode']).agg(['count']).Comment.plot(kind='bar')
ax.set_yscale('log')
plt.ylabel("Log(Count)")
plt.xlabel('ExitCode')
plt.show()

In [ ]:
#scatter_matrix(data, alpha=0.2, figsize=(6, 6), diagonal='kde')